<a href="https://colab.research.google.com/github/Ryan-M-Smith/CS315/blob/main/InClass/pi_monte_carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numba import njit, prange
import datetime
import random
import tensorflow as tf

In [9]:
# Naive approach

start_time = datetime.datetime.now()

def is_inside(x: float, y: float) -> bool:
  return x ** 2 + y ** 2 <= 1.0

n = 10000000
inside = 0
for i in range(n):
  x, y = random.random(), random.random()
  inside += x ** 2 + y ** 2 <= 1.0

pi_est = 4.0 * inside / n
print(f"π ≈ {pi_est}")

end_time = datetime.datetime.now()
elapsed_time = (end_time - start_time).total_seconds()

print(f"Completed simulation in {elapsed_time}s")



π ≈ 3.141596
Completed simulation in 5.537432s


In [10]:
# Just in time compilation into machine code
@njit(parallel=True)
def monte_carlo_pi(n):
  inside = 0
  for i in prange(n):
    x = random.random()
    y = random.random()
    if x**2+y**2<=1.0:
      inside +=1
  return inside

n = 10000000
start_time = datetime.datetime.now()
inCircle=monte_carlo_pi(n)
end_time = datetime.datetime.now()
elapsed_time = (end_time - start_time).total_seconds()
pi_est_parallel = 4.0*inCircle/n
print(pi_est_parallel,elapsed_time)

3.1425052 2.35935


In [11]:
# Numpy approach, parallel vectors and hardware optimization, based on https://rse.shef.ac.uk/hpc-intro-tuos-citc/16-parallel/index.html
n = 10000000
#start_time = datetime.datetime.now()

x = np.random.uniform(size=n)
y = np.random.uniform(size=n)
start_time = datetime.datetime.now()
radii = np.sqrt(x*x + y*y)
#counting how many inside the circle
count = len(radii[np.where(radii<=1.0)])

end_time = datetime.datetime.now()
elapsed_time = (end_time - start_time).total_seconds()
pi_est_numpy = 4.0 * count / n

# Since the largest variables in the script are x, y, and radii, each containing n points,
# and each is stored as a NumPy float64, to calculate the total memory required we can
# use NumPy’s dtype function to calculate the size of a float64.
size_of_float = np.dtype(np.float64).itemsize
memory = 3 * n * size_of_float / (1024**3)

print(pi_est_numpy,elapsed_time," memory used, in gigs: ",memory)

3.1418224 0.230753  memory used, in gigs:  0.22351741790771484
